## Escrevendo um Analisador Sintático para a Linguagem uChuck

Nesta etapa, você deve escrever uma versão preliminar de um analisador sintático para a linguagem uChuck. A especificação da gramática do uChuck em BNF está [aqui](https://colab.research.google.com/drive/1GiV8weG5lzVvA7z970EiE8bUMj3DAg0x?usp=sharing). Sua tarefa é escrever regras de análise sintática usando o SLY. Para um melhor entendimento, estude o capítulo [“Escrevendo um Analisador Sintático”](https://sly.readthedocs.io/en/latest/sly.html#writing-a-parser)
da documentação do SLY.

### Especificação
Sua tarefa é traduzir as regras listadas em uma gramática BNF em uma coleção de métodos decorados pelo decorador `@_()`. O nome de cada método deve corresponder ao nome da regra gramatical que está sendo analisada. O argumento para o decorador `@_()` é uma cadeia de caracteres que descreve o lado direito da gramática. Assim, uma regra gramatical como:

```
    <program> ::= <statement_list> EOF
```

torna-se um método de classe do Python da forma:

```python
class UChuckParser(Parser):
    """A parser for the uChuck language."""
    ...
    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return ('program', p.statement_list)
```

Para construir uma árvore de sintaxe, basta criar e retornar uma tupla ou lista em cada função de regra gramatical, como mostrado acima.

Seu objetivo, ao final deste segundo projeto, é reconhecer **sintaticamente** programas expressos na linguagem uChuck.
Para isso, o ideal é que você faça com que sua gramática não apresente **nenhum** conflito empilha/reduz.

**Sugestão:** Você deve começar de forma simples e trabalhar incrementalmente até construir a gramática completa.

### Esboço do Analisador Sintático

In [1]:
import sys
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UChuckLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1FXsbf3mp9rf-Cce-c9qk_8S43ppcPd4k?usp=sharing) e cole na célula abaixo.

In [3]:
class UChuckLexer(Lexer):
    """A lexer for the uChuck language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func

    # Reserved keywords
    keywords = {
        'int': "INT",
        'float': "FLOAT",
        'if': "IF",
        'else': "ELSE",
        'while': "WHILE",
        'break': "BREAK",
        'continue': "CONTINUE",
        'true': "TRUE",
        'false': "FALSE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
        # Identifiers
        "ID",
        # Constants
        "FLOAT_VAL",
        "INT_VAL",
        "STRING_LIT",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "PERCENT",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NEQ",
        "AND",
        "OR",
        "EXCLAMATION",
        # Assignment
        "CHUCK",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACE",
        "RBRACE",  # { }
        "L_HACK",
        "R_HACK",  # <<< >>>
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'\n+'
    ignore_comment = r'/\*(.|\n)*?\*/|//.*'

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    FLOAT_VAL = r'\d+\.\d+|\d+\.\d*|\.\d+'
    INT_VAL = r'\d+'
    STRING_LIT = r'"([^"\\]|\\.)*"'
    
    unterm_string = r'"(\\["\\]|[^"\\])*'
    unterm_comment = r'/\*.*'

    L_HACK = r'<<<'
    R_HACK = r'>>>'

    LE = r'<='
    GE = r'>='
    EQ = r'=='
    NEQ = r'!='
    AND = r'&&'
    OR = r'\|\|'
    CHUCK = r'=>'

    LT = r'<'
    GT = r'>'
    PLUS = r'\+'
    MINUS = r'-'
    TIMES = r'\*'
    DIVIDE = r'/'
    PERCENT = r'%'
    EXCLAMATION = r'!'

    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r','
    SEMI = r';'

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
        self.lineno += len(t.value)

    def ignore_comment(self, t):
        self.lineno += t.value.count("\n")

    def find_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        return token.index - last_cr

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_column(token)

    def unterm_comment(self, t):
        msg = "Unterminated comment"
        self._error(msg, t)

    def unterm_string(self, t):
        msg = "Unterminated string literal"
        self._error(msg, t)

    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

In [4]:
class UChuckParser(Parser):
    """A parser for the uChuck language."""
    
    # Get the token list from the lexer (required)
    tokens = UChuckLexer.tokens

    precedence = (
        ('right', 'CHUCK'),
        ('left', 'OR'),
        ('left', 'AND'),
        ('left', 'EQ', 'NEQ'),
        ('left', 'LT', 'LE', 'GT', 'GE'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'TIMES', 'DIVIDE', 'PERCENT'),
        ('right', 'EXCLAMATION'),
        ('right', 'UMINUS'),  # This is a virtual token for unary minus
    )

    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        self.lexer = UChuckLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    def _token_coord(self, p):
        return p.lineno, self.lexer.find_column(p)

    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return ('program', p.statement_list)

    # <statement_list> ::= { <statement> }+
    @_('statement statement_list')
    def statement_list(self, p):
        return [p.statement] + p.statement_list
    
    @_('statement')
    def statement_list(self, p):
        return [p.statement]

    # <statement> ::= <expression_statement>
    #               | <loop_statement>
    #               | <selection_statement>
    #               | <jump_statement>
    #               | <code_segment>
    @_('expression_statement',
       'loop_statement',
       'selection_statement',
       'jump_statement',
       'code_segment')
    def statement(self, p):
        return p[0]

    # <jump_statement> ::= "break" ";"
    #                    | "continue" ";"
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return 'break @ %d:%d' % self._token_coord(p)
    
    @_('CONTINUE SEMI')
    def jump_statement(self, p):
        return 'continue @ %d:%d' % self._token_coord(p)

    # <selection_statement> ::= "if" "(" <expression> ")" <statement> { "else" <statement> }?
    @_('IF LPAREN expression RPAREN statement ELSE statement')
    def selection_statement(self, p):
        return ('if @ %d:%d' % self._token_coord(p), p.expression, p.statement0, p.statement1)
    
    @_('IF LPAREN expression RPAREN statement')
    def selection_statement(self, p):
        return ('if @ %d:%d' % self._token_coord(p), p.expression, p.statement, None)

    # <loop_statement> ::= "while" "(" <expression> ")" <statement>
    @_('WHILE LPAREN expression RPAREN statement')
    def loop_statement(self, p):
        return ('while @ %d:%d' % self._token_coord(p), p.expression, p.statement)

    # <code_segment> ::= "{" { <statement_list> }? "}"
    @_('LBRACE statement_list RBRACE')
    def code_segment(self, p):
        return ('stmt_list @ %d:%d' % self._token_coord(p), p.statement_list)
    
    @_('LBRACE RBRACE')
    def code_segment(self, p):
        return ('stmt_list @ %d:%d' % self._token_coord(p), [])

    # <expression_statement> ::= { <expression> }? ";"
    @_('expression SEMI')
    def expression_statement(self, p):
        return ('expr', p.expression)
    
    @_('SEMI')
    def expression_statement(self, p):
        return ('expr', None)

    # <expression> ::= <chuck_expression> { "," <chuck_expression> }*
    @_('chuck_expression COMMA expression')
    def expression(self, p):
        if isinstance(p.expression, tuple) and p.expression[0] == 'expr_list':
            return ('expr_list', [p.chuck_expression] + p.expression[1])
        else:
            return ('expr_list', [p.chuck_expression, p.expression])
    
    @_('chuck_expression')
    def expression(self, p):
        return p.chuck_expression

    # <chuck_expression> ::= { <chuck_expression> "=>" }? <decl_expression>
    @_('chuck_expression CHUCK decl_expression')
    def chuck_expression(self, p):
        return ('chuck_op @ %d:%d' % self._token_coord(p), p.decl_expression, p.chuck_expression)
    
    @_('decl_expression')
    def chuck_expression(self, p):
        return p.decl_expression

    # <decl_expression> ::= <binary_expression>
    #                     | <type_decl> <identifier>
    @_('binary_expression')
    def decl_expression(self, p):
        return p.binary_expression
    
    @_('type_decl identifier')
    def decl_expression(self, p):
        return ('var_decl', *p.type_decl, *p.identifier)

    # <type_decl> ::= "int"
    #               | "float"
    #               | <identifier>
    @_('INT')
    def type_decl(self, p):
        return ('type: int @ %d:%d' % self._token_coord(p),)
    
    @_('FLOAT')
    def type_decl(self, p):
        return ('type: float @ %d:%d' % self._token_coord(p),)

    @_('identifier')
    def type_decl(self, p):
        return ('type: %s @ %d:%d' % (p.identifier[0].split()[1], *self._token_coord(p)),)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression> "+"  <binary_expression>
    #                       | <binary_expression> "-"  <binary_expression>
    #                       | <binary_expression> "*"  <binary_expression>
    #                       | <binary_expression> "/"  <binary_expression>
    #                       | <binary_expression> "%"  <binary_expression>
    #                       | <binary_expression> "<=" <binary_expression>
    #                       | <binary_expression> "<"  <binary_expression>
    #                       | <binary_expression> ">=" <binary_expression>
    #                       | <binary_expression> ">"  <binary_expression>
    #                       | <binary_expression> "==" <binary_expression>
    #                       | <binary_expression> "!=" <binary_expression>
    #                       | <binary_expression> "&&" <binary_expression>
    #                       | <binary_expression> "||" <binary_expression>
    @_('binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression',
       'binary_expression TIMES binary_expression',
       'binary_expression DIVIDE binary_expression',
       'binary_expression PERCENT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression LT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NEQ binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression')
    def binary_expression(self, p):
        return ('binary_op: %s @ %d:%d' % (p[1], *self._token_coord(p)), p.binary_expression0, p.binary_expression1)
    
    @_('unary_expression')
    def binary_expression(self, p):
        return p.unary_expression

    # <unary_expression> ::= <primary_expression>
    #                      | <unary_operator> <unary_expression>
    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return ('unary_op: %s @ %d:%d' % (p.unary_operator[0], *self._token_coord(p)), p.unary_expression)
    
    @_('MINUS unary_expression %prec UMINUS')
    def unary_expression(self, p):
        return ('unary_op: - @ %d:%d' % self._token_coord(p), p.unary_expression)
    
    @_('primary_expression')
    def unary_expression(self, p):
        return p.primary_expression

    # <unary_operator> ::= "+"
    #                    | "!"
    @_('PLUS')
    def unary_operator(self, p):
        return ('+ @ %d:%d' % self._token_coord(p),)
    
    @_('EXCLAMATION')
    def unary_operator(self, p):
        return ('! @ %d:%d' % self._token_coord(p),)

    # <primary_expression> ::= <literal>
    #                        | <location>
    #                        | "<<<" <expression> ">>>"
    #                        | "(" <expression> ")"
    @_('literal')
    def primary_expression(self, p):
        return p.literal
    
    @_('location')
    def primary_expression(self, p):
        return p.location
    
    @_('L_HACK expression R_HACK')
    def primary_expression(self, p):
        return ('print @ %d:%d' % self._token_coord(p), p.expression)
    
    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return p.expression

    @_('INT_VAL')
    def literal(self, p):
        return 'literal: int, %s @ %d:%d' % (p.INT_VAL, *self._token_coord(p))
    
    @_('FLOAT_VAL')
    def literal(self, p):
        return 'literal: float, %s @ %d:%d' % (p.FLOAT_VAL, *self._token_coord(p))
    
    @_('STRING_LIT')
    def literal(self, p):
        return 'literal: string, %s @ %d:%d' % (p.STRING_LIT, *self._token_coord(p))
    
    @_('TRUE')
    def literal(self, p):
        return 'literal: int, 1 @ %d:%d' % self._token_coord(p)
    
    @_('FALSE')
    def literal(self, p):
        return 'literal: int, 0 @ %d:%d' % self._token_coord(p)

    # <location> ::= <identifier>
    @_('identifier')
    def location(self, p):
        id_text = p.identifier[0]
        parts = id_text.split()
        name = parts[1]
        coord = ' '.join(parts[3:])
        return 'location: %s @ %s' % (name, coord)

    # <identifier> ::= ID
    @_('ID')
    def identifier(self, p):
        return ('id: %s @ %d:%d' % (p.ID, *self._token_coord(p)),)

In [5]:
def build_tree(root):
    return '\n'.join(_build_tree(root))

def _build_tree(node):
    if isinstance(node, list):
        if not node: return
        node = tuple(node)

    if not isinstance(node, tuple):
        yield " "+str(node)
        return

    values = [_build_tree(n) for n in node]
    if len(values) == 1:
        yield from build_lines('──', '  ', values[0])
        return

    start, *mid, end = values
    yield from build_lines('┬─', '│ ', start)
    for value in mid:
        yield from build_lines('├─', '│ ', value)
    yield from build_lines('└─', '  ', end)

def build_lines(first, other, values):
    try:
        yield first + next(values)
        for value in values:
            yield other + value
    except StopIteration:
        return

def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)

def main(args):
    parser = UChuckParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        st = parser.parse(f.read())
        if st is not None:
            print(build_tree(st))

## Teste
Para o desenvolvimento inicial, tente executar o analisador sintático em um arquivo de entrada de exemplo, como:

```
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}
```

In [6]:
%%file test.uck
/* print values of factorials */
1 => int n;
1 => int value;

while( n < 10 )
{
	value * n => value;
	<<< value >>>;
	n + 1 => n;
}

Writing test.uck


E o resultado será semelhante ao texto mostrado abaixo.

```
┬─ program
└─┬─┬─ expr
  │ └─┬─ chuck_op @ 2:1
  │   ├─┬─ var_decl
  │   │ ├─ type: int @ 2:6
  │   │ └─ id: n @ 2:10
  │   └─ literal: int, 1 @ 2:1
  ├─┬─ expr
  │ └─┬─ chuck_op @ 3:1
  │   ├─┬─ var_decl
  │   │ ├─ type: int @ 3:6
  │   │ └─ id: value @ 3:10
  │   └─ literal: int, 1 @ 3:1
  └─┬─ while @ 5:1
    ├─┬─ binary_op: < @ 5:8
    │ ├─ location: n @ 5:8
    │ └─ literal: int, 10 @ 5:12
    └─┬─ stmt_list @ 6:1
      └─┬─┬─ expr
        │ └─┬─ chuck_op @ 7:2
        │   ├─ location: value @ 7:15
        │   └─┬─ binary_op: * @ 7:2
        │     ├─ location: value @ 7:2
        │     └─ location: n @ 7:10
        ├─┬─ expr
        │ └─┬─ print @ 8:2
        │   └─ location: value @ 8:6
        └─┬─ expr
          └─┬─ chuck_op @ 9:2
            ├─ location: n @ 9:11
            └─┬─ binary_op: + @ 9:2
              ├─ location: n @ 9:2
              └─ literal: int, 1 @ 9:6
```

In [7]:
main(["test.uck"])

┬─ program
└─┬─┬─ expr
  │ └─┬─ chuck_op @ 2:1
  │   ├─┬─ var_decl
  │   │ ├─ type: int @ 2:6
  │   │ └─ id: n @ 2:10
  │   └─ literal: int, 1 @ 2:1
  ├─┬─ expr
  │ └─┬─ chuck_op @ 3:1
  │   ├─┬─ var_decl
  │   │ ├─ type: int @ 3:6
  │   │ └─ id: value @ 3:10
  │   └─ literal: int, 1 @ 3:1
  └─┬─ while @ 5:1
    ├─┬─ binary_op: < @ 5:8
    │ ├─ location: n @ 5:8
    │ └─ literal: int, 10 @ 5:12
    └─┬─ stmt_list @ 6:1
      └─┬─┬─ expr
        │ └─┬─ chuck_op @ 7:2
        │   ├─ location: value @ 7:15
        │   └─┬─ binary_op: * @ 7:2
        │     ├─ location: value @ 7:2
        │     └─ location: n @ 7:10
        ├─┬─ expr
        │ └─┬─ print @ 8:2
        │   └─ location: value @ 8:6
        └─┬─ expr
          └─┬─ chuck_op @ 9:2
            ├─ location: n @ 9:11
            └─┬─ binary_op: + @ 9:2
              ├─ location: n @ 9:2
              └─ literal: int, 1 @ 9:6


## Anexo
A lista abaixo define os nós da árvore de sintaxe que devem ser retornados em cada regra da gramática:

```
program = tuple('program', statement_list)

statement_list = list(statement)

statement = expression_statement
          | loop_statement
          | selection_statement
          | jump_statement
          | code_segment

jump_statement = tuple('break @ lineno:column')
               | tuple('continue @ lineno:column')

selection_statement = tuple('if @ lineno:column', expression, statement0, statement1)

loop_statement = tuple('while @ lineno:column', expression, statement)

code_segment = tuple('stmt_list @ lineno:column', statement_list)

expression_statement = tuple('expr', expression)

expression = chuck_expression
           | tuple('expr_list', list(chuck_expression))

chuck_expression = tuple('chuck_op @ lineno:column', decl_expression, chuck_expression)
                 | decl_expression

decl_expression = binary_expression
                | tuple('var_decl', type_decl, tuple('id: ' + str(ID) + ' @ lineno:column'))

type_decl = tuple('type: int @ lineno:column')
          | tuple('type: float @ lineno:column')
          | tuple('type: ' + str(ID) + ' @ lineno:column')

binary_expression = unary_expression
                  | tuple('binary_op: + @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: - @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: * @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: / @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: % @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: <= @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: < @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: >= @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: > @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: == @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: != @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: && @ lineno:column', binary_expression0, binary_expression1)
                  | tuple('binary_op: || @ lineno:column', binary_expression0, binary_expression1)

unary_expression = primary_expression
                 | tuple('unary_op: ' + str(unary_operator), unary_expression)

unary_operator = tuple('+ @ lineno:column')
               | tuple('- @ lineno:column')
               | tuple('! @ lineno:column')

primary_expression = literal
                   | location
                   | tuple('print @ lineno:column', expression)
                   | expression

literal = tuple('literal: int, ' + str(INT_VAL) + ' @ lineno:column')
        | tuple('literal: float, ' + str(FLOAT_VAL) + ' @ lineno:column')
        | tuple('literal: string, ' + str(STRING_LIT) + ' @ lineno:column')
        | tuple('literal: int, 1 @ lineno:column')
        | tuple('literal: int, 0 @ lineno:column')

location = tuple('location: ' + str(ID) + ' @ lineno:column')
```

Um novo nó é criado sempre que o valor retornado for uma tupla do Python, por exemplo:

```python
    # <program> ::= <statement_list> EOF
    @_('statement_list')
    def program(self, p):
        return ('program', p.statement_list)
```

Uma lista de nós é criada sempre que o valor retornado for uma lista do Python, por exemplo:

```python
    # <statement_list> ::= { <statement> }+
    @_('statement { statement }')
    def statement_list(self, p):
        return [p.statement0] + p.statement1
```

Uma referência para um nó é criada sempre que o valor retornado em uma regra for o nome de outra regra.

Os valores indicados como `lineno` e `column` referem-se aos números de linha e coluna, respectivamente, do símbolo terminal mais à esquerda de cada produção, se houver.